In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import numpy as np
import pandas as pd

In [4]:
# Step 1: Utilize VGG16 for Feature Extraction
# Load VGG16 model pre-trained on ImageNet, excluding the top layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional layers of VGG16
for layer in vgg_base.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [1]:
# Step 2: Data Preprocessing and Augmentation
# Setup data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'data/cats_and_dogs/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'data/cats_and_dogs/validation',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
# Step 3: Build and Train the Classification Model
# Add custom fully connected layers on top of the VGG16 base model
model = models.Sequential()
model.add(vgg_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Step 4: Model Compilation and Training
# Compile the model
model.compile(optimizer=optimizers.SGD(lr=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

ValueError: Argument(s) not recognized: {'lr': 0.001}

In [ ]:
# Step 5: Evaluate and Test the Model
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_generator, steps=50)
print(f'Validation accuracy: {val_acc}')

In [ ]:
# Save the model
model.save('cat_dog_classifier_vgg16.h5')

In [ ]:
# Prediction function to classify new images
def predict_image(image_path):
    from tensorflow.keras.preprocessing import image
    import numpy as np

    img = image.load_img(image_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.0

    prediction = model.predict(img_tensor)
    if prediction[0] > 0.5:
        print(f'Prediction: Dog with confidence {prediction[0]:.2f}')
    else:
        print(f'Prediction: Cat with confidence {1 - prediction[0]:.2f}')